In [384]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
import numpy as np
from nltk import word_tokenize, FreqDist
import string
from os import path
from PIL import Image
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
# from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer as sa
# from nltk.classify.scikitlearn import SklearnClassifier
# from nltk.classify import ClassifierI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.model_selection import train_test_split, cross_validate
reader= Reader(rating_scale=(-5.0, 5.0))
from surprise import Dataset, Reader, accuracy, SVD
from surprise.prediction_algorithms import SVD, KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV
import requests

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

with open('userratingsDF.pickle', 'rb') as f:
    dfr = pickle.load(f)
with open('CategoricalItemDF.pickle', 'rb') as f:
    full = pickle.load(f)
with open('_SENT&RATINGDF.pickle', 'rb') as f:
    df1 = pickle.load(f)
with open('recdf.pickle', 'rb') as f:
    rec = pickle.load(f)
with open('ratingsmeandf.pickle', 'rb') as f:
    ratemean = pickle.load(f)

In [318]:
toadd=full[['url','prodName']]

In [319]:
rec.rename(columns={'prodName':'brandName'},inplace=True)

In [320]:
rec=rec.merge(toadd,on='url').drop_duplicates()
rec=rec.merge(ratemean,on='url')
rec.rename(columns={'rating_x':'rating'},inplace=True)
rec.drop(columns='rating_y',inplace=True)
rec.head()

,brandName,rating,url,user,prodName,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,60


In [322]:
reader = Reader()
data = Dataset.load_from_df(rec[['user','url','rating']],reader)

In [324]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  1378 

Number of items:  514


## SVD/Grid Search ???!!!??!?!?!

In [335]:
params = {'n_factors' :[10,20,30,40,50,60,70,80,90,100,200],
         'reg_all':[0.02,0.05,0.1,0.5]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

/Users/mackenziemitchell/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [336]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.9636548214343789, 'mae': 0.6639878362747851}
{'rmse': {'n_factors': 70, 'reg_all': 0.1}, 'mae': {'n_factors': 70, 'reg_all': 0.02}}


## Seeing what classification model is best/Cross Validation !!!??!?1/!?!?!??!?!


In [337]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)

In [338]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.83406781, 0.95413201, 0.93217687, 1.14529959, 1.00957739]))
('test_mae', array([0.62510757, 0.66341568, 0.65852128, 0.75986052, 0.68958191]))
('fit_time', (0.0511019229888916, 0.04958796501159668, 0.04799604415893555, 0.055422067642211914, 0.05212593078613281))
('test_time', (0.002173185348510742, 0.0023238658905029297, 0.0020699501037597656, 0.0022420883178710938, 0.0021560192108154297))
-----------------------
0.975050731600222


In [345]:
# cross validating with KNNBasic
nknn_basic = KNNBasic(sim_options={'name':'pearson','user_based':False})
ncv_knn_basic= cross_validate(nknn_basic,data,n_jobs=-1)

In [346]:
for i in ncv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(ncv_knn_basic['test_rmse']))

('test_rmse', array([0.93043739, 1.09713101, 0.8761668 , 1.02863049, 0.94821475]))
('test_mae', array([0.65870953, 0.73356431, 0.63586002, 0.69981893, 0.66337086]))
('fit_time', (0.01104593276977539, 0.009692907333374023, 0.00887298583984375, 0.008912801742553711, 0.009455204010009766))
('test_time', (0.0028519630432128906, 0.0020720958709716797, 0.00191497802734375, 0.0018391609191894531, 0.0019881725311279297))
-----------------------
0.9761160896774775


In [344]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.90899229, 0.91428427, 0.86416265, 1.019977  , 1.11242536]))
('test_mae', array([0.65004662, 0.65338825, 0.63150447, 0.68702091, 0.73408563]))
('fit_time', (0.041764020919799805, 0.03380703926086426, 0.0325620174407959, 0.03156590461730957, 0.03202033042907715))
('test_time', (0.0020961761474609375, 0.0021262168884277344, 0.0020198822021484375, 0.0021021366119384766, 0.0019991397857666016))


0.9639683120139555

In [347]:
# cross validating with KNNBaseline
nknn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':False})
ncv_knn_baseline = cross_validate(nknn_baseline,data)
for i in ncv_knn_baseline.items():
    print(i)

np.mean(ncv_knn_baseline['test_rmse'])

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.95128352, 0.89507063, 0.90778273, 1.11707132, 0.94462029]))
('test_mae', array([0.66732507, 0.64196982, 0.63807522, 0.7341412 , 0.65966026]))
('fit_time', (0.015108823776245117, 0.01285099983215332, 0.012653827667236328, 0.012573003768920898, 0.014127016067504883))
('test_time', (0.0027451515197753906, 0.0020499229431152344, 0.0019960403442382812, 0.001961946487426758, 0.0019099712371826172))


0.963165698563273

## Description TFIDF and Cosine Similarity !?!??!?!?!??!

In [350]:
df1['description']=df1['title']+df1['content']

In [351]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df1['description'])

In [352]:
tfidf_matrix.shape

(1685, 39382)

In [353]:
full.shape

(1701, 30)

In [354]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [403]:
cosine_sim

array([[1.        , 0.03513622, 0.02979474, ..., 0.00730602, 0.        ,
        0.00399307],
       [0.03513622, 1.        , 0.02854128, ..., 0.        , 0.        ,
        0.        ],
       [0.02979474, 0.02854128, 1.        , ..., 0.        , 0.        ,
        0.00425702],
       ...,
       [0.00730602, 0.        , 0.        , ..., 1.        , 0.        ,
        0.05258715],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.00154483],
       [0.00399307, 0.        , 0.00425702, ..., 0.05258715, 0.00154483,
        1.        ]])

In [356]:
# def item(id):  
#     return df1.loc[df1['url'] == id]['description'].tolist()
# # Just reads the results out of the dictionary.def 
# def recommend(item_id, num):
#     print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
#     print("-------")    
#     recs = results[item_id][:num]   
#     for rec in recs: 
#         print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [357]:
# df1.reset_index(inplace=True)
# products = df1['url']
# indices = pd.Series(df1.index, index=df1['url'])

In [358]:
# def get_recommendations(product):
#     idx = indices[products]
#     sim_scores = list(enumerate(cosine_sim[idx]))
# #     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]
#     prod_indices = [i[0] for i in sim_scores]
#     return products.iloc[prod_indices]

In [359]:
get_recommendations(11429304).drop_duplicates()

1     10236357
5     10305907
7     10305956
11    10306437
15    10306558
25    10362821
28    10362879
Name: url, dtype: object

In [360]:
# recommend(10362879,5)

In [361]:
alldf=dfr[['url','prodName','rating','user']]

In [381]:
alldf.groupby(['url','user','prodName']).mean().head()

rating
url      user      prodName        
10236357 adrian    Anthony        5
         george    Anthony        5
         kimber    Anthony        5
         mike      Anthony        5
         westtexas Anthony        5

In [382]:
alldf.sort_values(['url','user']).head()

,url,prodName,rating,user
1587,10236357,Anthony,5,adrian
1589,10236357,Anthony,5,george
1586,10236357,Anthony,5,kimber
1588,10236357,Anthony,5,mike
1585,10236357,Anthony,5,westtexas


In [383]:
alldf.sort_values('user').head()

,url,prodName,rating,user
3247,11289064,PCA SKIN,5,000000
321,11294885,Juice Beauty,2,1908
1426,11287696,Glytone,5,2
1827,11054488,First Aid Beauty,5,24fan
1637,11054488,First Aid Beauty,5,24fan


In [365]:
# data=Dataset.load_from_df(df1[['user','url','rating']],reader=reader)
# data


In [366]:
data=Dataset.load_from_df(alldf[['user','url','rating']],reader=reader)
data

In [370]:
data=Dataset.load_from_df(rec[['user','url','rating']],reader=reader)
data

In [371]:
train, test = train_test_split(data,test_size=0.2)

In [372]:
sim_cos = {'name':'cosine','user_based':False}

In [392]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [393]:
basic.sim
predictions = basic.test(test)
print(accuracy.rmse(predictions))

RMSE: 1.0327
1.032656202946828


In [394]:
sim_cos_user = {'name':'cosine','user_based':True}

In [395]:
basic = knns.KNNBasic(sim_options=sim_cos_user)
basic.fit(train)
basic.sim
predictions = basic.test(test)
print(accuracy.rmse(predictions))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0343
1.034265688040528


In [396]:
sim_pearson = {'name':'pearson','user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(train)
predictions = basic_pearson.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0322
1.032213853270103


In [397]:
sim_pearson_user = {'name':'pearson','user_based':True}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson_user)
basic_pearson.fit(train)
predictions = basic_pearson.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0322
1.032213853270103


In [398]:
sim_pearson = {'name':'pearson','user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(train)
predictions = knn_means.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0252
1.0251658879801462


In [399]:
sim_pearson_us = {'name':'pearson','user_based':True}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson_user)
knn_means.fit(train)
predictions = knn_means.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0786
1.0785894485989567


In [400]:
sim_pearson = {'name':'pearson','user_based':False}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(train)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0109
1.0108791109869182


In [401]:
sim_pearson_use = {'name':'pearson','user_based':True}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson_user)
knn_baseline.fit(train)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0109
1.0108791109869182


In [402]:
predictions[5][3]

4.228397168312853

## More Grid Search??

In [202]:
param_grid = {'n_factors':[20,100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  66 out of  80 | elapsed:    2.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    3.3s finished


In [203]:
svd = SVD(n_factors=100,n_epochs=10,lr_all=0.005,reg_all=0.4)
svd.fit(train)
predictions = svd.test(test)
print(accuracy.rmse(predictions))

RMSE: 0.9242
0.9241714166225277


In [204]:
user_34_prediction = svd.predict('3','20')
user_34_prediction

Prediction(uid='3', iid='20', r_ui=None, est=4.55314371257485, details={'was_impossible': False})

In [58]:
df=dfr[['user','rating','url','prodName']]

In [59]:
df.sample(1)[['url','prodName']]

,url,prodName
1677,10941238,Erno Laszlo


In [137]:
rec

,brandName,rating,url,user,prodName
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...
6,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...
12,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...
18,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...
24,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...
30,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...
36,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...
42,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...
48,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...
54,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel


## Does this produce recommendations or is it messed up?

In [169]:
# def skincare_rater(df,num, typ=None, problem=None):
#     userID = 1000
#     rating_list = []
#     while num > 0:
#         if typ:
#             prod = df[df[''].str.contains(typ)].sample(1)
#         else:
#             sample=df.sample(1)
#             brand=sample[['brandName']]
#             product=sample[['prodName']]
#             url=sample[['url']]
#             prod = sample[['brandName','prodName','url']]
#         print(prod)
#         rating = input('How do you rate this product on a scale of 1-5, press n if you have not used :\n')
#         if rating == 'n':
#             continue
#         else:
#             rating_one_prod=df.sample(1)[['url','prodName']]
#             rating_one_prod['user']=1000
#             rating_one_prod['rating']=rating
#             ratedict={'url':url,'brandName':brand,'prodName':product}
#             rating_list.append(rating_one_prod) 
#             num -= 1
#     return rating_list

# :) Collaborative Filtering Rec System

In [196]:
def movie_rater(df,num, typ=None):
    userID = 1000
    rating_list = []
    while num > 0:
#         if typ:
#             prod = df[df['genres'].str.contains(genre)].sample(1)
#         else:
        prod = df.sample(1)
        print(prod)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_prod = {'userId':userID,'url':prod[['url']],'brandName':prod[['brandName']],'prodName':prod[['prodName']],'rating':rating}
            rating_list.append(rating_one_prod) 
#             num -= 1
#     return rating_list

In [218]:
rec.rename({'rating_x':'rating'},inplace=True)

In [247]:
def skin_rater(movie_df,num, typ=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if typ:
            p = movie_df[movie_df['types'].str.contains(genre)].sample(1)
        else:
            p = movie_df.sample(1)
        print(p[['brandName','prodName','url']])
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'user':userID,'url':p['url'].values[0],'brand':p['brandName'].values[0],'product':p['prodName'].values[0],'rating_x':rating,'rating_count':p['rating_count'].values[0]}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list

In [271]:
user_rating=skin_rater(rec,3)

         brandName                                           prodName  \
1371  The Ordinary  The Ordinary Niacinamide 10% + Zinc 1% High St...   

           url  
1371  11363398  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
         brandName                                           prodName  \
370  SkinCeuticals  SkinCeuticals Sheer Physical UV Defense SPF50 ...   

          url  
370  11907698  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
2
    brandName                                   prodName       url
486    NuFACE  NuFACE Gel Primer 24K Gold Complex - Firm  11288879
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


In [272]:
reader = Reader()
data = Dataset.load_from_df(rec[['user','url','rating_x']],reader=reader)

In [273]:
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(data.build_full_trainset())

In [274]:
list_of_prods = []
for m_id in rec['url'].unique():
    list_of_prods.append( (m_id,svd_.predict(1000,m_id)[3]))
ranked_prods = sorted(list_of_prods,key=lambda x:x[1],reverse=True)

In [275]:
user_ratings=pd.DataFrame(user_rating)

In [291]:
def recommended_products(user_ratings,df,n):
    for idx, re in enumerate(user_ratings):
        title = df.loc[df['url'] == int(re[0])]['prodName']
        u=re[0]
        
        print('Recommendation # ',idx+1,': ', 'url:', u, rec[rec['url']==u]['prodName'].drop_duplicates(),'\n')
        n-= 1
        if n == 0:
            break
recommended_products(ranked_prods,rec,5)

Recommendation #  1 :  url: 10543756 1561    Dr. Brandt Pores No More Pore Refiner (1oz)
Name: prodName, dtype: object 

Recommendation #  2 :  url: 11524843 1485    FOREO LUNA™ 2 for Oily Skin
Name: prodName, dtype: object 

Recommendation #  3 :  url: 11286360 1061    Avène Thermal Spring Water 10.58oz
Name: prodName, dtype: object 

Recommendation #  4 :  url: 11286857 598    Cellex-C Advanced C Serum
Name: prodName, dtype: object 

Recommendation #  5 :  url: 11524845 1140    FOREO LUNA™ mini 2 - Fuchsia
Name: prodName, dtype: object 

